In [0]:
#data-set-url: "https://www.kaggle.com/c/fake-news/data?select=train.csv"

import pandas as pd
import tensorflow as tf
import numpy as np

In [0]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot  
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

In [0]:
from google.colab import files

In [16]:
upload = files.upload()

Saving train.csv to train.csv


In [0]:
#df = pd.read_csv(root_dir+"/"+"train.csv")
#from google.colab import drive
#drive.mount('C:/Users/Khushi Raj/Desktop/Anaconda-projects/NLP_projects/fake_News_classifier_using_Lstm/fake-news/')
df = pd.read_csv("train.csv")

In [0]:
df.head()

In [0]:
# drop Nan values
df = df.dropna()

In [0]:
# Get the Independent Features
X = df.drop("label", axis=1)

In [0]:
# Get the Dependent Features
y = df["label"]

In [0]:
X.shape

In [0]:
y.shape

In [0]:
tf.__version__

In [0]:
## Vocabulary size
voc_size = 5000

In [0]:
# Onehot Representation
messages = X.copy()

In [0]:
messages.reset_index(inplace=True)

In [0]:
import nltk
import re
from nltk.corpus import stopwords

In [0]:
nltk.download("stopwords")

In [0]:
## Data Processing

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  print(i)
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [0]:
corpus

In [0]:
onehot_rep = [one_hot(words, voc_size) for words in corpus]
onehot_rep

In [0]:
# Embedding Representation

sent_length = 20
embedded_docs = pad_sequences(onehot_rep, padding='pre', maxlen=sent_length)
print(embedded_docs)

In [0]:
embedded_docs[0]

In [0]:
len(embedded_docs)

In [0]:
## Creating Model

embedding_vector_feature = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_feature, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [0]:
len(embedded_docs), y.shape

In [0]:
# use numpy

X_final = np.array(embedded_docs)
y_final = np.array(y)

In [0]:
X_final.shape, y_final.shape

In [0]:
# using sklearn
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [0]:
## Model Training
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

In [0]:
# Performance metrics And Accuracy
y_pred = model.predict_classes(X_test)

In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
confusion_matrix(y_test, y_pred)

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)